In [120]:
import hashlib
import binascii
from bip_utils import (
    Bip39SeedGenerator, Bip32Slip10Secp256k1, Bip32Slip10Ed25519,
    Bip39MnemonicGenerator, Bip39Languages, Bip39WordsNum, Bip39MnemonicValidator, Bip39Mnemonic,
    Bip44Changes, Bip84Coins, Bip84
)
from pytezos.crypto.encoding import base58_encode
from pytezos.crypto import key
import pytezos
from eth_account import Account
from bitcoin.wallet import CBitcoinSecret, CBech32BitcoinAddress, P2WPKHBitcoinAddress, CBitcoinAddress
from bitcoin.core.key import CECKey, CPubKey
from bitcoin.core.script import CScript,OP_0
from bitcoin.core import Hash160
import bitcoin
import requests

# HD Wallets(public key, private key, address)

## Mnemonics

### Generate random mnemonic of twelve words

In [2]:
random_mnemonics = Bip39MnemonicGenerator(Bip39Languages.ENGLISH).FromWordsNumber(Bip39WordsNum.WORDS_NUM_12).ToStr()
random_mnemonics

'purse year quality awake paddle feel canvas cat bike profit maze this'

### Validate Mnemonics

In [3]:
mnemonic = "invalid abandon abandon abandon abandon abandon abandon abandon abandon abandon abandon about"

# Get if a mnemonic is valid with automatic language detection, return bool
valid = Bip39MnemonicValidator(Bip39Languages.ENGLISH).IsValid(mnemonic)
# Same but specifying the language
not_valid = Bip39MnemonicValidator(Bip39Languages.ENGLISH).IsValid(random_mnemonics)

valid, not_valid

(False, True)

In [4]:
# mnemonic = "imitate embody law mammal exotic transfer roof hope price swift ordinary uncle"
mnemonic = 'chat style uncover beauty federal dish priority best then empty lesson flag'
seed_bytes = Bip39SeedGenerator(mnemonic).Generate()

In [5]:
TEZOS_PATH = "m/44'/1729'/0'/0'"
ETHEREUM_PATH = "m/44'/60'/0'/0/0"
BITCOIN_PATH_LEGACY = "m/44'/0'/0'/0/0"
BITCOIN_PATH_SEGWIT = "m/84'/0'/0'/0/0"

### Tezos Key Store

In [32]:
bip32_ctx = Bip32Slip10Ed25519.FromSeedAndPath(seed_bytes, TEZOS_PATH)

In [33]:
private_key_hex = bip32_ctx.PrivateKey().Raw().ToHex()

In [34]:
tezos_private_key = base58_encode(bytes.fromhex(private_key_hex), prefix=b'edsk')
tezos_private_key

b'edsk2xWEMfyhLzafUqvgRVx3PsjNkbZJpCuiVCeMyK8DK78S4SNJxd'

In [35]:
tezos_account = key.Key.from_encoded_key(tezos_private_key)
tezos_address = tezos_account.public_key_hash()
tezos_address

'tz1SzAmjGiyrW2joqwbHBo1jeNGHrjXoE5tq'

### Ethereum Key Store

In [27]:
bip32_ctx = Bip32Slip10Secp256k1.FromSeedAndPath(seed_bytes, ETHEREUM_PATH)

In [28]:
private_key_hex = bip32_ctx.PrivateKey().Raw().ToHex()

In [29]:
ethereum_account = Account.from_key(private_key_hex)
ethereum_private_key = ethereum_account.privateKey
ethereum_private_key

HexBytes('0xab4a0d5fcafaa7fb9fbe59690e0ae2ee6c94989a5c07da79c40bd152583d6900')

In [31]:
ethereum_address = ethereum_account.address
ethereum_address

'0x1DEF760Eb8f81167dF93E27D8963a27895E3458f'

### Bitcoin Key Store

In [14]:
bip32_ctx = Bip32Slip10Secp256k1.FromSeedAndPath(seed_bytes, BITCOIN_PATH_SEGWIT)

In [15]:
private_key_bytes = bip32_ctx.PrivateKey().Raw().ToBytes()
private_key_bytes

b'9\xd3{k\x05O!\xe4\xf5F\x07\xc6\x0bD\x8b\xd7\xb5\xc2d \\\x92\xb4I\x81\xc4\xa43An.\x9b'

In [16]:
bitcoin_account = CBitcoinSecret.from_secret_bytes(private_key_bytes)

In [17]:
script_pubkey = CScript([OP_0, Hash160(bitcoin_account.pub)])
bitcoin_address = CBech32BitcoinAddress.from_scriptPubKey(script_pubkey)
bitcoin_address

P2WPKHBitcoinAddress('bc1q3qwydrdrq4tsfcf2yp5u55ja423p0wx6ss5mhu')

In [18]:
account.hex()

'39d37b6b054f21e4f54607c60b448bd7b5c264205c92b44981c4a433416e2e9b01'

# Nodes, Indexers, Explorers

## Block Explorers

Bitcoin, Ethereum, etc ...- https://blockchair.com/

Bitcoin - https://blockstream.info/

Ethereum - https://etherscan.io/

Tezos - https://tzkt.io/

## Indexers

Bitcoin - https://github.com/Blockstream/electrs

Ethereum - https://etherscan.io/

Tezos - https://api.tzkt.io/

In [41]:
# Tezos https://api.ghostnet.tzkt.io

r = requests.get(f'https://api.ghostnet.tzkt.io/v1/accounts/{tezos_address}/balance')
balance = int(r.text)/1000000

r = requests.get(f'https://api.ghostnet.tzkt.io/v1/accounts/{tezos_address}/operations')
history = r.json()

In [42]:
balance, history

(89.999256,
 [{'type': 'transaction',
   'id': 67253966995456,
   'level': 1326979,
   'timestamp': '2022-10-13T08:55:05Z',
   'block': 'BM7AWg6YmHFqCy3WyZ5Ymu3GM7RgfkoSqZ4x2MPD6akXvHRSwoZ',
   'hash': 'ooj8pWRHSbvmGAwbkSbk3JFCTqE7T83UP7iYWRJhADHRC7UJpdN',
   'counter': 12228198,
   'sender': {'address': 'tz1SzAmjGiyrW2joqwbHBo1jeNGHrjXoE5tq'},
   'gasLimit': 1101,
   'gasUsed': 1001,
   'storageLimit': 100,
   'storageUsed': 0,
   'bakerFee': 374,
   'storageFee': 0,
   'allocationFee': 0,
   'target': {'address': 'tz1R5X6pDpoxbMkY4AQJmUhRfEHjYN7dq66E'},
   'amount': 10000000,
   'status': 'applied',
   'hasInternals': False},
  {'type': 'reveal',
   'id': 67253841166336,
   'level': 1326976,
   'timestamp': '2022-10-13T08:54:20Z',
   'block': 'BKskGfrvCLAm3s3PmGAfAWf5LTA4jAs9hWcpas5pG7XTnee7mrH',
   'hash': 'op1G62wZsNDGU1BjqTURqUr5EmrBG5Y2VfCX7KCgnJQ9pxuNXXi',
   'sender': {'address': 'tz1SzAmjGiyrW2joqwbHBo1jeNGHrjXoE5tq'},
   'counter': 12228197,
   'gasLimit': 1000,
   'gasUsed':

In [43]:
# Ethereum  https://api-sepolia.etherscan.io

r = requests.get(f'https://api-sepolia.etherscan.io/api?module=account&action=balance&address=0x382b4ca2c4a7cd28c1c400c69d81ec2b2637f7dd&tag=latest')
balance = int(r.json()['result'])/1000000000000000000

r = requests.get(f'https://api-sepolia.etherscan.io/api?module=account&action=txlist&address=0x382b4ca2c4a7cd28c1c400c69d81ec2b2637f7dd&startblock=0&endblock=99999999&page=1&offset=10&sort=asc')
history = r.json()
# https://sepolia.etherscan.io/ https://sepolia-faucet.pk910.de/

In [23]:
balance, history

(21.50238249997988,
 {'status': '0',
  'message': 'NOTOK',
  'result': 'Max rate limit reached, please use API Key for higher rate limit'})

In [24]:
# Bitcoin https://blockstream.info/testnet/api


## Nodes

### Ethereum

- https://infura.io
- https://www.alchemy.com/

### Tezos

- https://mainnet.api.tez.ie
- https://mainnet.smartpy.io	
- https://rpc.tzbeta.net/	

# Sending Crypto

Sending crypto requires 3 steps

1. Build transaction

2. Sign transaction

3. Broadcast transaction

## Tezos

In [36]:
pytezos = pytezos.pytezos.using(shell="https://ghostnet.smartpy.io", key=tezos_account)

In [37]:
pytezos.reveal().autofill().sign().inject()

{'chain_id': 'NetXnHfVqm9iesp',
 'hash': 'op1G62wZsNDGU1BjqTURqUr5EmrBG5Y2VfCX7KCgnJQ9pxuNXXi',
 'protocol': 'PtKathmankSpLLDALzWw7CGD2j2MtyveTwboEYokqUCP4a1LxMg',
 'branch': 'BM8Dj1Shj52XGeTQY1y17U3mmVjGBs8UpB4xWKAdd24wLmBqFps',
 'contents': [{'kind': 'reveal',
   'source': 'tz1SzAmjGiyrW2joqwbHBo1jeNGHrjXoE5tq',
   'fee': '370',
   'counter': '12228197',
   'gas_limit': '1000',
   'storage_limit': '0',
   'public_key': 'edpkuCAjDLjv1JnJNJndFWjyCRvQqQniJ2CQ4xtoQdAYsbzRo7nrSi'}],
 'signature': 'sigw4RkqZCwTDVBK9FrQVgbArWRFbbstJzpn4oX3S1BXvEVjnE9Vm5DTkaYm4PuFeXBT3WoLJK2d7Uz5GAkzBVuDWSCHhdSX'}

In [39]:
tx = pytezos.transaction(destination='tz1R5X6pDpoxbMkY4AQJmUhRfEHjYN7dq66E', amount=10*1000000).autofill().sign().inject()

In [132]:
tx

{'chain_id': 'NetXnHfVqm9iesp',
 'hash': 'onuBmZ13KRw8CNYa6tu79vBMAQKQMeDbNcbRpYRPh7x4mqTPbax',
 'protocol': 'PtKathmankSpLLDALzWw7CGD2j2MtyveTwboEYokqUCP4a1LxMg',
 'branch': 'BMTQ21L3Ke46YtYYTH2gg2zq4iyRUBV8nhv91A1E5YY462gYTuf',
 'contents': [{'kind': 'transaction',
   'source': 'tz1grSQDByRpnVs7sPtaprNZRp531ZKz6Jmm',
   'fee': '373',
   'counter': '393388',
   'gas_limit': '1101',
   'storage_limit': '100',
   'amount': '10000000',
   'destination': 'tz1R5X6pDpoxbMkY4AQJmUhRfEHjYN7dq66E'}],
 'signature': 'sigsvGjkMX1P8zZh7r6gtTMxL7J9y4B2VWJHdEwyzMPCs7qDRGHT1QmbJWnSqw8rgd43jW9NSZ58wkosuutLFcfMMPitsfcF'}

In [40]:
tx['hash']

'ooj8pWRHSbvmGAwbkSbk3JFCTqE7T83UP7iYWRJhADHRC7UJpdN'

## Ethereum

In [56]:
web3 = web3.Web3(Web3.HTTPProvider('https://sepolia.infura.io/v3/82de4c56f4364dd899635d8ebbc349cc'))

In [57]:
web3.eth.get_block('latest')

AttributeDict({'baseFeePerGas': 7,
 'difficulty': 0,
 'extraData': HexBytes('0x'),
 'gasLimit': 30000000,
 'gasUsed': 53934,
 'hash': HexBytes('0xfdba95949e5354b1e8976f11364f7a3f03dafdc074013be360e29bf3c94db991'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'miner': '0xFf58d746A67C2E42bCC07d6B3F58406E8837E883',
 'mixHash': HexBytes('0x69ae7b82f344ae575b9430c7f58e317fcddfee31aebe2d5f81de569ecec33e67'),
 'nonce': HexBytes('0x0000000000000000'),
 'number': 2079158,
 'parentHash': HexBytes('0xfd51d9b

In [113]:
nonce = web3.eth.get_transaction_count(ethereum_address)

In [114]:
tr = {
    'to': web3.toChecksumAddress('0x9cBdFe782E9aa4cefcC572b90a6a8F5422AD22E8'),
    'value': w3.to_wei(0.001, 'gwei'),
    'gasPrice': web3.to_wei(5, 'gwei'), 
    'nonce': nonce,
    'gas': 100000,
    'chainId': 11155111
}

In [115]:
signed_tx = web3.eth.account.sign_transaction(tr, ethereum_private_key)

In [116]:
signed_tx

SignedTransaction(rawTransaction=HexBytes('0xf86c0285012a05f200830186a0949cbdfe782e9aa4cefcc572b90a6a8f5422ad22e8830f4240808401546d71a099ab2a9b7c9ceedc8b7b07f1ad6ceceae26faba17f8290392f33148d6832d6dea07e76c408ec992ca58c6a1ef59ce5607745aa7445aa1cb7c7c7ba9054bf5f4dc8'), hash=HexBytes('0x34c0acb5eb4f5ba817e2f24db44ffb50fd2576a327ccce07308148152003071e'), r=69506290746578442433779168540379508436569397901387647719406124350074145396446, s=57201259858015671701172849162525924246440631533984305242652112475247016365512, v=22310257)

In [117]:
tx_hash = web3.eth.send_raw_transaction(signed_tx.rawTransaction)

In [118]:
tx_hash

HexBytes('0x34c0acb5eb4f5ba817e2f24db44ffb50fd2576a327ccce07308148152003071e')

In [106]:
ethereum_address

'0x1DEF760Eb8f81167dF93E27D8963a27895E3458f'